In [3]:
import numpy as np
import pandas as pd

In [4]:
df_flights = pd.read_csv("filtered_flight_data.csv")

In [ ]:
df_flights.head(25)

In [ ]:
print(df_flights['DESTINATION_AIRPORT'].unique())


In [ ]:
#Keep only rows where Destination is not purely numeric
df = df_flights[~df_flights['DESTINATION_AIRPORT'].astype(str).str.isnumeric()]
df

In [ ]:
# Drop rows with missing tail numbers (for turnaround time analysis)
df_cleaned = df[~df['TAIL_NUMBER'].isna()]

# Fill missing values in delay cause columns with 0
delay_cols = ['AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 
              'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']
df_cleaned[delay_cols] = df_cleaned[delay_cols].fillna(0)


In [9]:
df_cleaned.to_csv("data.csv", index=False)

In [ ]:
df_cleaned.isnull().sum()

In [ ]:
df_cleaned.dtypes

In [10]:
df_cleaned = df_cleaned.copy()
cols_to_fill = [
    'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF',
    'ELAPSED_TIME', 'AIR_TIME', 'WHEELS_ON', 'TAXI_IN',
    'ARRIVAL_TIME', 'ARRIVAL_DELAY'
]
df_cleaned[cols_to_fill] = df_cleaned[cols_to_fill].fillna(0)

# Fill object columns with 'Unknown'
object_cols_to_fill = ['CANCELLATION_REASON', 'CANCELLATION_DESCRIPTION']
df_cleaned[object_cols_to_fill] = df_cleaned[object_cols_to_fill].fillna(np.nan)

In [11]:
df_cleaned.isnull().sum()

YEAR                            0
MONTH                           0
DAY                             0
DAY_OF_WEEK                     0
FLIGHT_NUMBER                   0
TAIL_NUMBER                     0
DESTINATION_AIRPORT             0
SCHEDULED_DEPARTURE             0
DEPARTURE_TIME                  0
DEPARTURE_DELAY                 0
TAXI_OUT                        0
WHEELS_OFF                      0
SCHEDULED_TIME                  0
ELAPSED_TIME                    0
AIR_TIME                        0
DISTANCE                        0
WHEELS_ON                       0
TAXI_IN                         0
SCHEDULED_ARRIVAL               0
ARRIVAL_TIME                    0
ARRIVAL_DELAY                   0
DIVERTED                        0
CANCELLED                       0
CANCELLATION_REASON         90286
AIR_SYSTEM_DELAY                0
SECURITY_DELAY                  0
AIRLINE_DELAY                   0
LATE_AIRCRAFT_DELAY             0
WEATHER_DELAY                   0
AIRLINE_y     

In [12]:
df_cleaned.to_csv("final_dataset.csv", index=False)

Identify delay patterns by day of week

In [13]:
# Calculate average arrival delay by day of the week
delay_by_day = df_cleaned.groupby("DAY_OF_WEEK")["ARRIVAL_DELAY"].mean().reset_index()

delay_by_day.columns = ["Day of Week", "Average Arrival Delay (min)"]
delay_by_day.sort_values("Day of Week", inplace=True)
delay_by_day


,Day of Week,Average Arrival Delay (min)
0,1,6.709370
1,2,4.858753
2,3,4.238298
3,4,5.776337
4,5,5.045984
5,6,2.237754
6,7,3.988979


Analyze the impact of weather on delays.

In [14]:
# Analyze the relationship between weather delays and overall arrival delays
weather_impact = df_cleaned[["WEATHER_DELAY", "ARRIVAL_DELAY"]]

# Calculate how many flights had weather-related delays
weather_delay_flights = weather_impact[weather_impact["WEATHER_DELAY"] > 0]

# Summary statistics
weather_delay_summary = {
    "Total Flights": len(df_cleaned),
    "Flights with Weather Delay": len(weather_delay_flights),
    "Percentage with Weather Delay": round(len(weather_delay_flights) / len(df_cleaned) * 100, 2),
    "Average Weather Delay (all flights)": round(df_cleaned["WEATHER_DELAY"].mean(), 2),
    "Average Weather Delay (affected flights only)": round(weather_delay_flights["WEATHER_DELAY"].mean(), 2),
    "Average Arrival Delay (affected by weather)": round(weather_delay_flights["ARRIVAL_DELAY"].mean(), 2),
}

weather_delay_summary


{'Total Flights': 91545,
 'Flights with Weather Delay': 1074,
 'Percentage with Weather Delay': 1.17,
 'Average Weather Delay (all flights)': np.float64(0.54),
 'Average Weather Delay (affected flights only)': np.float64(45.75),
 'Average Arrival Delay (affected by weather)': np.float64(82.54)}

Calculate overall on-time performance rate.

In [15]:
on_time_flights = df_cleaned[df_cleaned["ARRIVAL_DELAY"] <= 0]
on_time_rate = round(len(on_time_flights) / len(df_cleaned) * 100, 2)

on_time_rate

63.32

Find delay trends by time of day (morning, afternoon, evening).

In [19]:
def categorize_time_of_day(hour):
    if 5 <= hour < 12:
        return "Morning"
    elif 12 <= hour < 17:
        return "Afternoon"
    elif 17 <= hour < 21:
        return "Evening"
    else:
        return "Night"

# Ensure you're working with the right DataFrame
df_cleaned.loc[:, "SCHED_DEP_HOUR"] = df_cleaned["SCHEDULED_DEPARTURE"] // 100
df_cleaned.loc[:, "TIME_OF_DAY"] = df_cleaned["SCHED_DEP_HOUR"].apply(categorize_time_of_day)

# Group and analyze
delay_by_time = df_cleaned.groupby("TIME_OF_DAY")["ARRIVAL_DELAY"].mean().reset_index()
delay_by_time.columns = ["Time of Day", "Average Arrival Delay (min)"]
delay_by_time.sort_values("Time of Day", inplace=True)

print(delay_by_time)


  Time of Day  Average Arrival Delay (min)
0   Afternoon                     6.696021
1     Evening                     9.910425
2     Morning                     0.159656
3       Night                     7.065947


Detect seasonal trends in delay rates (monthly/quarterly).

In [21]:
df_cleaned["FLIGHT_DATE"] = pd.to_datetime(df_cleaned[["YEAR", "MONTH", "DAY"]])


In [24]:
# Monthly average arrival delay
monthly_delay = df_cleaned.groupby("MONTH")["ARRIVAL_DELAY"].mean().reset_index()
monthly_delay.columns = ["Month", "Avg Arrival Delay (min)"]
monthly_delay["Month_Name"] = monthly_delay["Month"].apply(lambda x: pd.to_datetime(str(x), format="%m").strftime("%B"))
monthly_delay = monthly_delay.sort_values("Month")

# Quarterly delay trend
df_cleaned["QUARTER"] = df_cleaned["FLIGHT_DATE"].dt.to_period("Q")
quarterly_delay = df_cleaned.groupby("QUARTER")["ARRIVAL_DELAY"].mean().reset_index()
quarterly_delay.columns = ["Quarter", "Avg Arrival Delay (min)"]

print("Monthly Delay Trends:")
print(monthly_delay)

print("\nQuarterly Delay Trends:")
print(quarterly_delay)


Monthly Delay Trends:
    Month  Avg Arrival Delay (min) Month_Name
0       1                 5.164329    January
1       2                 7.958012   February
2       3                 4.707563      March
3       4                 3.183250      April
4       5                 4.035449        May
5       6                 8.831544       June
6       7                 7.140310       July
7       8                 4.662294     August
8       9                -0.137250  September
9      11                 0.561303   November
10     12                 6.233636   December

Quarterly Delay Trends:
  Quarter  Avg Arrival Delay (min)
0  2015Q1                 5.861688
1  2015Q2                 5.375112
2  2015Q3                 4.015441
3  2015Q4                 3.436254


Analyze turnaround time between flights per aircraft (if tail numbers available).

In [27]:
df_turnaround = df_cleaned.dropna(subset=["TAIL_NUMBER", "ARRIVAL_TIME", "SCHEDULED_DEPARTURE"])
df_turnaround["FLIGHT_DATE"] = pd.to_datetime(df_turnaround[["YEAR", "MONTH", "DAY"]])

# Step 2: Convert ARRIVAL_TIME and SCHEDULED_DEPARTURE to proper 4-digit strings
df_turnaround["ARRIVAL_TIME"] = df_turnaround["ARRIVAL_TIME"].astype(int).astype(str).str.zfill(4)
df_turnaround["SCHEDULED_DEPARTURE"] = df_turnaround["SCHEDULED_DEPARTURE"].astype(int).astype(str).str.zfill(4)

# Step 3: Combine to full datetime
df_turnaround["ARRIVAL_DATETIME"] = pd.to_datetime(df_turnaround["FLIGHT_DATE"].astype(str) + " " + df_turnaround["ARRIVAL_TIME"], format="%Y-%m-%d %H%M", errors="coerce")
df_turnaround["DEP_DATETIME"] = pd.to_datetime(df_turnaround["FLIGHT_DATE"].astype(str) + " " + df_turnaround["SCHEDULED_DEPARTURE"], format="%Y-%m-%d %H%M", errors="coerce")

# Step 4: Sort and calculate turnaround
df_turnaround = df_turnaround.sort_values(["TAIL_NUMBER", "FLIGHT_DATE", "DEP_DATETIME"])
df_turnaround["PREV_ARRIVAL"] = df_turnaround.groupby("TAIL_NUMBER")["ARRIVAL_DATETIME"].shift(1)
df_turnaround["TURNAROUND_MIN"] = (df_turnaround["DEP_DATETIME"] - df_turnaround["PREV_ARRIVAL"]).dt.total_seconds() / 60

# Step 5: Filter reasonable turnarounds
df_turnaround_filtered = df_turnaround[(df_turnaround["TURNAROUND_MIN"] > 0) & (df_turnaround["TURNAROUND_MIN"] < 1000)]

# Final average
avg_turnaround = df_turnaround_filtered["TURNAROUND_MIN"].mean()
print(f"✅ Average Turnaround Time Between Flights: {avg_turnaround:.2f} minutes")



✅ Average Turnaround Time Between Flights: 427.53 minutes
